<a href="https://colab.research.google.com/github/susmita99-collab/Summer-Analytics-2025-Assignment/blob/main/Dynamic_Pricing_Final_Fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

# 📦 Step 0: Install required libraries
!pip install pathway bokeh panel --quiet

# 🧠 Step 1: Import libraries
import numpy as np
import pandas as pd
import pathway as pw
import datetime
import bokeh.plotting
import panel as pn

# 📥 Step 2: Load and preprocess data
# Upload your 'dataset.csv' to Colab before running this
df = pd.read_csv('dataset.csv')

# ✅ Fix: Create Timestamp column with correct datetime format
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort and save required columns to a new CSV for streaming
df = df.sort_values('Timestamp').reset_index(drop=True)
df[['Timestamp', 'Occupancy', 'Capacity', 'QueueLength', 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType']].to_csv("parking_stream.csv", index=False)

# 📐 Step 3: Define schema for Pathway
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: int
    IsSpecialDay: int
    VehicleType: str

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

# 🕓 Step 4: Add time columns
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# 🧮 Step 5: Demand-based pricing logic
def vehicle_type_weight(vehicle):
    weights = {"car": 1.0, "bike": 0.5, "truck": 1.5}
    return weights.get(vehicle.lower(), 1.0)

extended = data_with_time.with_columns(
    demand = (
        0.4 * (data_with_time.Occupancy / data_with_time.Capacity)
        + 0.2 * data_with_time.QueueLength
        - 0.2 * data_with_time.TrafficConditionNearby
        + 0.1 * data_with_time.IsSpecialDay
        + 0.1 * pw.apply(vehicle_type_weight, data_with_time.VehicleType)
    )
)

# 💰 Step 6: Normalize demand and calculate price
BASE_PRICE = 10
MAX_MULTIPLIER = 2.0
MIN_MULTIPLIER = 0.5

windowed = (
    extended.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t = pw.this._pw_window_end,
        demand_mean = pw.reducers.mean(pw.this.demand)
    )
    .with_columns(
        price = pw.apply(
            lambda d: max(MIN_MULTIPLIER, min(MAX_MULTIPLIER, 1 + d)) * BASE_PRICE,
            pw.this.demand_mean
        )
    )
)

# 📊 Step 7: Visualize using Bokeh
pn.extension()

def price_plotter(source):
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Dynamic Parking Price (Daily)",
        x_axis_type="datetime",
    )
    fig.line("t", "price", source=source, line_width=2, color="green")
    fig.circle("t", "price", source=source, size=6, color="red")
    return fig

viz = windowed.plot(price_plotter, sorting_col="t")
pn.Column(viz).servable()

# 🚀 Step 8: Run the Pathway stream processor
%%capture --no-display
pw.run()

TypeError: Pathway does not support using binary operator mul on columns of types <class 'float'>, typing.Any.
It refers to the following expression:
	(0.1 * pathway.apply(vehicle_type_weight, <table1>.VehicleType))
called in /tmp/ipython-input-3-4258632274.py:49
with tables:
	<table1> created in /tmp/ipython-input-3-4258632274.py:39
